In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import datetime


In [2]:
df = pd.read_csv('nyc bus data.csv')
df = df.drop_duplicates()
# = df.dropna()
df.head()

C:\Users\sajee\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,Unnamed: 17
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,40.635170,-73.960803,FOSTER AV/E 18 ST,approaching,76,6/1/2017 0:03,24:06:14,NaN
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,40.590802,-74.158340,MERRYMOUNT ST/TRAVIS AV,approaching,62,6/1/2017 0:03,23:58:02,NaN
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,40.886010,-73.912647,HENRY HUDSON PKY E/W 235 ST,at stop,5,6/1/2017 0:03,24:00:53,NaN
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,40.668002,-73.729348,HOOK CREEK BL/SUNRISE HY,< 1 stop away,267,6/1/2017 0:04,24:03:00,NaN
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,40.868134,-73.893032,GRAND CONCOURSE/E 196 ST,at stop,11,6/1/2017 0:03,23:59:38,NaN


In [3]:
df.shape

(1048575, 18)

In [4]:
#subset the dataframe to include only the first 100,000 rows, or observations.
df = df.iloc[:100000, :]
df.shape

(100000, 18)

In [5]:
#checking for column types and missing data
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 18 columns):
RecordedAtTime               100000 non-null object
DirectionRef                 100000 non-null int64
PublishedLineName            100000 non-null object
OriginName                   100000 non-null object
OriginLat                    100000 non-null float64
OriginLong                   100000 non-null float64
DestinationName              100000 non-null object
DestinationLat               100000 non-null float64
DestinationLong              100000 non-null float64
VehicleRef                   100000 non-null object
VehicleLocation.Latitude     100000 non-null float64
VehicleLocation.Longitude    100000 non-null float64
NextStopPointName            99904 non-null object
ArrivalProximityText         99904 non-null object
DistanceFromStop             99904 non-null object
ExpectedArrivalTime          88630 non-null object
ScheduledArrivalTime         98312 non-null object
Unname

In [6]:
df.isnull().sum()

RecordedAtTime                   0
DirectionRef                     0
PublishedLineName                0
OriginName                       0
OriginLat                        0
OriginLong                       0
DestinationName                  0
DestinationLat                   0
DestinationLong                  0
VehicleRef                       0
VehicleLocation.Latitude         0
VehicleLocation.Longitude        0
NextStopPointName               96
ArrivalProximityText            96
DistanceFromStop                96
ExpectedArrivalTime          11370
ScheduledArrivalTime          1688
Unnamed: 17                  99995
dtype: int64

In [7]:
#creating new features by converting strings to datetimes
df['expected_arrival'] = pd.to_datetime(df['ExpectedArrivalTime'], errors="coerce")
df['scheduled_arrival'] = pd.to_datetime(df['ScheduledArrivalTime'], errors="coerce")

In [8]:
#checking for null values before filling in missing data
df['expected_arrival'].isnull().value_counts()

False    88625
True     11375
Name: expected_arrival, dtype: int64

In [9]:
#checking for null values before filling in missing data
df['scheduled_arrival'].isnull().value_counts()

False    93530
True      6470
Name: scheduled_arrival, dtype: int64

In [10]:
#re-checking the shape of the data
df.shape

(100000, 20)

In [11]:
#filling in missing data 
df['expected_arrival'].bfill(axis=0, inplace=True)
df['scheduled_arrival'].bfill(axis=0, inplace=True)
df['DistanceFromStop'].ffill(axis=0, inplace=True)
df['ArrivalProximityText'].ffill(axis=0, inplace=True)
df['NextStopPointName'].ffill(axis=0, inplace=True)
df['ExpectedArrivalTime'].bfill(axis=0, inplace=True)
df['ScheduledArrivalTime'].bfill(axis=0, inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 20 columns):
RecordedAtTime               100000 non-null object
DirectionRef                 100000 non-null int64
PublishedLineName            100000 non-null object
OriginName                   100000 non-null object
OriginLat                    100000 non-null float64
OriginLong                   100000 non-null float64
DestinationName              100000 non-null object
DestinationLat               100000 non-null float64
DestinationLong              100000 non-null float64
VehicleRef                   100000 non-null object
VehicleLocation.Latitude     100000 non-null float64
VehicleLocation.Longitude    100000 non-null float64
NextStopPointName            100000 non-null object
ArrivalProximityText         100000 non-null object
DistanceFromStop             100000 non-null object
ExpectedArrivalTime          100000 non-null object
ScheduledArrivalTime         100000 non-null object
U

In [13]:
print(df['expected_arrival'].head())

0   2017-06-01 00:03:00
1   2017-06-01 00:03:00
2   2017-06-01 00:03:00
3   2017-06-01 00:04:00
4   2017-06-01 00:03:00
Name: expected_arrival, dtype: datetime64[ns]


In [14]:
#correcting the year value in the two new feature columns
df['expected_arrival'] = df['expected_arrival'].apply(lambda t: t.replace(year=2017) if not pd.isnull(t) else pd.NaT)
df['scheduled_arrival'] = df['scheduled_arrival'].apply(lambda t: t.replace(year=2017) if not pd.isnull(t) else pd.NaT)

#correcting the day value in the new feature columns
df['expected_arrival'] = df['expected_arrival'].apply(lambda t: t.replace(day=1) if not pd.isnull(t) else pd.NaT)
df['scheduled_arrival'] = df['scheduled_arrival'].apply(lambda t: t.replace(day=1) if not pd.isnull(t) else pd.NaT)

df['expected_arrival'] = df['expected_arrival'].apply(lambda t: t.replace(month=6) if not pd.isnull(t) else pd.NaT)
df['scheduled_arrival'] = df['scheduled_arrival'].apply(lambda t: t.replace(month=6) if not pd.isnull(t) else pd.NaT)
#print(df['expected_arrival'])

In [15]:
#create new feature to record the difference between each bus's scheduled arrival time and expected arrival time.
df['time_difference'] = df['scheduled_arrival'] - df['expected_arrival']

In [16]:
df.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,...,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,Unnamed: 17,expected_arrival,scheduled_arrival,time_difference
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,...,-73.960803,FOSTER AV/E 18 ST,approaching,76,6/1/2017 0:03,24:06:14,NaN,2017-06-01 00:03:00,2017-06-01 23:58:02,23:55:02
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,...,-74.158340,MERRYMOUNT ST/TRAVIS AV,approaching,62,6/1/2017 0:03,23:58:02,NaN,2017-06-01 00:03:00,2017-06-01 23:58:02,23:55:02
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,...,-73.912647,HENRY HUDSON PKY E/W 235 ST,at stop,5,6/1/2017 0:03,24:00:53,NaN,2017-06-01 00:03:00,2017-06-01 23:59:38,23:56:38
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,...,-73.729348,HOOK CREEK BL/SUNRISE HY,< 1 stop away,267,6/1/2017 0:04,24:03:00,NaN,2017-06-01 00:04:00,2017-06-01 23:59:38,23:55:38
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,...,-73.893032,GRAND CONCOURSE/E 196 ST,at stop,11,6/1/2017 0:03,23:59:38,NaN,2017-06-01 00:03:00,2017-06-01 23:59:38,23:56:38


In [17]:
df.time_difference.dtype

dtype('<m8[ns]')

In [18]:
df['delay'] = df['time_difference'].dt.minute > 5

AttributeError: 'TimedeltaProperties' object has no attribute 'minute'

In [ ]:
#creating new feature column to indicate if a bus is on time or behind schedule
df['hour'] = df['ExpectedArrivalTime'].dt.hour
df['minute'] = df['ExpectedArrivalTime'].dt.minute

In [ ]:
#an unconventional way to check for the null, nan, or NaT value in scheduled_arrival
#df['checkNaN'] = df['scheduled_arrival'].isnull()
#print(np.where(df['checkNaN'] == True)) 